In [79]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../src/')
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, WDiscriminator
from off_manifolder import offmanifolder_maker
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit, GeodesicBridge, GeodesicFM
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset


import hydra
from omegaconf import DictConfig, OmegaConf
import wandb
from pytorch_lightning.loggers import WandbLogger, TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import ot as pot

adjoint = False
if adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint

In [2]:
import sys
sys.path.append('../src/')
from diffusion import DiffusionModel
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from model2 import Autoencoder, WDiscriminator

In [3]:
def guided_generate(dm, num_samples, guidance_grad, lam=1.):
    dm.eval()  # Ensure the model is in evaluation mode

    # Initial random noise
    x_t = torch.randn(num_samples, dm.hparams.data_size, device=dm.device)

    # Utilize the buffers for betas, alphas, and alphas_cumprod (already on the correct device)
    betas = dm.betas
    alphas = dm.alphas
    alphas_cumprod = dm.alphas_cumprod

    # Generate t values for reverse process, normalized
    t_values = torch.arange(0, dm.hparams.num_steps, device=dm.device).flip(0)

    with torch.no_grad():
        for t in t_values:
            # Calculate the current alpha and noise for this step
            sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod[t])
            sqrt_one_minus_alphas_cumprod = torch.sqrt(1.0 - alphas_cumprod[t])
            t_norm = t.float() / (dm.hparams.num_steps - 1)
            t_batch = t_norm.repeat(num_samples, 1)

            # Model prediction to refine the current state
            epsilon0_t = dm(x_t, t_batch)
            epsilon_t = epsilon0_t - lam * guidance_grad(x_t, t_batch)
            
            mu_t = (x_t - (1 - alphas[t]) / sqrt_one_minus_alphas_cumprod * epsilon_t) / torch.sqrt(alphas[t])
            noise1 = torch.randn(num_samples, dm.hparams.data_size, device=dm.device)
            x_t = mu_t + torch.sqrt(betas[t]) * noise1

    return x_t

In [23]:
# main_sweep_id = '59muwbsf'
main_sweep_id = 'jtpxi61p'
main_disc_sweep_id = '5nidtly5'
main_data_name = 'torus_none_0'
main_weights_cycle = 1.
main_dimensions_latent = 3

wandb.login()
api = wandb.Api()

entity = "xingzhis"
project = "dmae"
sweep_id = main_sweep_id
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

data_name = main_data_name
# run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == main_weights_cycle) & (df['dimensions.latent'] == main_dimensions_latent)]['run_id']
assert len(run_ids) == 1
run_id = run_ids.iloc[0]
run = api.run(f"{entity}/{project}/{run_id}")
# run = api.run(f"{entity}/{project}/{run_id}")
# folder_path = '../../src/wandb/'
folder_path = '../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# cfg.data.root = '../' + cfg.data.root
model = Autoencoder.load_from_checkpoint(ckpt_path)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    model.eval()
    x = torch.tensor(data['X'], dtype=torch.float32, device=model.device)
    z = model.encoder(x)
    xh = model.decoder(z)
    zhh = model.encoder(xh)
zc = z.cpu().numpy()
# fig = scatter(zc, s=2, alpha=0.2, title='latent', filename=f'{results_path}/latent.html')
# wandb.log({"latent": wandb.Plotly(fig)})
# fig = scatter(zhh, s=2, alpha=0.2, title='latent_reembedd', filename=f'{results_path}/latent_reembedded.html')
# wandb.log({"latent_reembedded": wandb.Plotly(fig)})
# fig = scatter(x, s=2, alpha=0.2, title='data', filename=f'{results_path}/data.html')
# wandb.log({"data": wandb.Plotly(fig)})
# fig = scatter(xh, s=2, alpha=0.2, title='reconstruction', filename=f'{results_path}/reconstruction.html')
# wandb.log({"reconstruction": wandb.Plotly(fig)})

sweep_id = main_disc_sweep_id
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
# Initialize an empty list to store run data
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)
run_ids = df[(df['data.name'] == data_name) & (df['cfg/training/clamp'] == 0.1)][['run_id']]
assert len(run_ids) == 1
run_id = run_ids.iloc[0]
run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
# folder_path = '../../src/wandb/'
folder_path = '../src/wandb/'
cfg = OmegaConf.create(run.config)
folder_list = glob.glob(f"{folder_path}*{run.id}*")
ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
ckpt_path = ckpt_files[0]
# cfg.data.root = '../' + cfg.data.root
wd = WDiscriminator.load_from_checkpoint(ckpt_path)
data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)
with torch.no_grad():
    wd.eval()
    x_disc = torch.tensor(data['data'], dtype=torch.float32, device=wd.device)
    probab = wd(x_disc).flatten()
    z = model.encoder(x_disc)

# title = f'{data_name} {noise_type} {noise_level} {dist_mask}'
# scatter(zc, data['mask_x'], s=2, alpha=0.2, title=title, filename=filename).show()
# scatter(zhh, data['mask_x'], s=2, alpha=0.2, title=title, filename=filename).show()
scatter(x_disc, data['mask_x'], s=2, alpha=0.2, title=f'{data_name}: input').show()
scatter(x_disc, probab, s=2, alpha=0.2, title=f'{data_name}: probability').show()#, filename='hsphere_disc_prob.html').show()
scatter(x_disc, probab > 0.5, s=2, alpha=0.2, title=f'{data_name}: classification').show()

# if plot_recon:
#     title = f'[RECON] {title}'
#     if savefolder is not None:
#         filename = f"{savefolder}/{title1}.html"
#     # scatter(xh, data['mask_x'], s=2, alpha=0.2, title=title1, filename=filename).show()
max_prob = probab.max()

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.

/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning:

Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.



In [24]:
jac = jacobian(model.encoder, x)
U, S, V = torch.linalg.svd(jac, full_matrices=False)

In [88]:
def logvol(x, f, k, eps=1e-7):
    jac = jacobian(f, x)
    U, S, V = torch.linalg.svd(jac, full_matrices=False)
    sk = S[...,:k]
    logsk = torch.log(sk + eps)
    logvols = logsk.sum(dim=-1)
    return logvols

def gradlogvol(x, f, k, eps=1e-7):
    return jacobian(lambda x: logvol(x, f, k, eps).unsqueeze(-1), x)

In [89]:
def dummy(x):
    return jacobian(f, x)[:,0,:]

In [90]:
jacobian(dummy, x).min()

tensor(-17.9992, grad_fn=<MinBackward1>)

In [92]:
gradlogvol(x, f, 2)

tensor([[[-0.6763, -0.0000, -7.5139]],

        [[-0.8389, -1.9029, -0.0000]],

        [[ 0.0000,  0.9782,  2.0099]],

        ...,

        [[-2.1258, -0.7085, -0.0000]],

        [[-0.8297, -1.6032, -0.0000]],

        [[-2.1544, -0.8055,  0.0000]]], grad_fn=<CopySlices>)

In [84]:
jac = jacobian(model.encoder, x)

In [30]:
jac.requires_grad

True

In [31]:
U, S, V = torch.linalg.svd(jac, full_matrices=False)

In [32]:
sk = S[...,:2]
logsk = torch.log(sk + 1e-7)
logvols = logsk.sum(dim=-1)

In [71]:
def f(x):
    return x ** 3

In [72]:
from torch.autograd import grad
create_graph = True
retain_graph = True
x = x.clone()
x.requires_grad_(True)
# model.no_grad()
output = f(x)
batch_size, output_dim, input_dim = *output.shape, x.shape[-1]

# Use autograd's grad function to get gradients for each output dimension
jacobian = torch.zeros(batch_size, output_dim, input_dim).to(x.device)
for i in range(output_dim):
    grad_outputs = torch.zeros(batch_size, output_dim).to(x.device)
    grad_outputs[:, i] = 1.0
    gradients = grad(outputs=output, inputs=x, grad_outputs=grad_outputs, create_graph=create_graph, retain_graph=retain_graph, only_inputs=True)[0]
    jacobian[:, i, :] = gradients

In [73]:
output2 = jacobian[:,0,:]

In [75]:
output_dim2 = output2.shape[1]
jacobian2 = torch.zeros(batch_size, output_dim2, input_dim).to(x.device)
for i in range(output_dim2):
    grad_outputs = torch.zeros(batch_size, output_dim2).to(x.device)
    grad_outputs[:, i] = 1.0
    gradients = grad(outputs=output2, inputs=x, grad_outputs=grad_outputs, create_graph=create_graph, retain_graph=retain_graph, only_inputs=True)[0]
    jacobian2[:, i, :] = gradients

In [76]:
torch.abs(jacobian2).sum()

tensor(26239.1328, grad_fn=<SumBackward0>)

In [70]:
torch.abs(jacobian).sum()

tensor(11727.8887, grad_fn=<SumBackward0>)

In [98]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.preprocessor = Preprocessor()
        self.mlp = MLP()


class Preprocessor(nn.Module):
    # Define your preprocessor details if needed
    def forward(self, x):
        # Preprocessing steps
        return x

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64, 3)
        )

    def forward(self, x):
        return self.net(x)

# Initialize the model
model2 = MLP()

# Set the model to evaluation mode
# model.eval()


In [99]:
gradlogvol(x, model2, 2)

tensor([[[ 0.4900, -0.1677,  0.0727]],

        [[-1.0154,  1.4034,  1.2467]],

        [[-0.9818, -1.5949, -1.4225]],

        ...,

        [[ 0.0899,  0.3370,  0.2517]],

        [[ 0.7990,  0.5488,  0.8597]],

        [[ 0.5580,  0.5813, -0.6366]]], grad_fn=<CopySlices>)

In [100]:
gradlogvol(x, model, 2)

tensor([[[0., 0., 0.]],

        [[0., 0., 0.]],

        [[0., 0., 0.]],

        ...,

        [[0., 0., 0.]],

        [[0., 0., 0.]],

        [[0., 0., 0.]]], grad_fn=<CopySlices>)

In [107]:
model.encoder.eval()
jacobian(model.encoder, x)

tensor([[[-0.1461, -0.1584,  1.0898],
         [-0.4582,  0.2801,  1.5950],
         [-0.0445, -0.7247,  0.1147]],

        [[ 0.1385, -0.3848,  0.1812],
         [-0.5573,  0.1437,  0.4448],
         [ 0.1102, -0.5591,  0.1923]],

        [[ 0.7168, -0.1920,  0.3472],
         [-0.4294,  0.0235,  0.2912],
         [-0.1041, -0.1881,  0.5100]],

        ...,

        [[ 0.5579, -0.4273,  1.0659],
         [-0.7325, -0.0590,  1.3860],
         [ 0.0056, -0.1106,  0.0882]],

        [[ 0.1243, -0.3764,  0.1932],
         [-0.5075,  0.1126,  0.4434],
         [ 0.1214, -0.5156,  0.2035]],

        [[ 0.5771, -0.1283,  0.4398],
         [-0.4088,  0.2714,  0.2897],
         [ 0.0245, -0.1226, -0.3620]]], grad_fn=<CopySlices>)

In [108]:
model.encoder.MLP

Encoder(
  (preprocessor): Preprocessor()
  (mlp): MLP(
    (net): Sequential(
      (0): Linear(in_features=3, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): Dropout(p=0.5, inplace=False)
      (8): Linear(in_features=128, out_features=64, bias=True)
      (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): Dropout(p=0.5, inplace=False)
      (12): Linear(in_features=64, out_features=3, bias=True)
    )
  )
)

In [140]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

class MLP2(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_hidden_layers):
        super(MLP2, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ELU()]
        
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

In [141]:
model3 = MLP2(3, 32, 2, 3)

In [142]:
jacobian(model3, x)

tensor([[[-2.8998e-02,  2.8073e-03,  8.2867e-03],
         [ 5.7347e-02, -2.1302e-02,  2.4328e-04]],

        [[-3.3347e-02,  5.2819e-03,  6.9675e-03],
         [ 4.8465e-02, -1.7028e-02,  2.0038e-02]],

        [[ 1.6544e-02,  4.8035e-05, -5.4196e-03],
         [ 2.9617e-02, -3.2869e-03, -2.8936e-02]],

        ...,

        [[-3.2602e-02,  1.1286e-02,  1.3538e-03],
         [ 4.1031e-02, -8.4615e-03,  2.3988e-02]],

        [[-3.3478e-02,  5.9801e-03,  6.7895e-03],
         [ 4.9754e-02, -1.9645e-02,  2.1770e-02]],

        [[-1.6529e-02,  3.0752e-03, -2.7675e-02],
         [ 2.2861e-02,  6.5425e-03, -3.4795e-02]]], grad_fn=<CopySlices>)

In [143]:
gradlogvol(x, model3, 2)

tensor([[[-0.1306,  0.0829, -0.1069]],

        [[ 0.1857, -0.0721, -0.0505]],

        [[-0.6565,  0.3961,  0.5053]],

        ...,

        [[-0.3134,  0.1165,  0.3614]],

        [[ 0.1824, -0.0721, -0.0576]],

        [[ 0.2887,  0.0313, -0.1302]]], grad_fn=<CopySlices>)

In [144]:
gradlogvol(x, f, 2)


tensor([[[-0.6763, -0.0000, -7.5139]],

        [[-0.8389, -1.9029, -0.0000]],

        [[ 0.0000,  0.9782,  2.0099]],

        ...,

        [[-2.1258, -0.7085, -0.0000]],

        [[-0.8297, -1.6032, -0.0000]],

        [[-2.1544, -0.8055,  0.0000]]], grad_fn=<CopySlices>)